In [3]:
import h5py
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img
from scipy.misc import imread, imresize
import imageio
import numpy as np
import PIL
import matplotlib.pyplot as plt

## data preprocessing and normalization

In [4]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## load pretrained model (99.25% CNN)

In [6]:
model = load_model('/Users/zetong/cnn.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

## classification demo

In [7]:
x = np.asarray(PIL.Image.open('/Users/zetong/mnist_png/testing/2/1802.png'))
img = PIL.Image.fromarray(x, mode='L')
x = x.reshape(28,28)
x = x.reshape(1,28,28,1)
x = x.astype('float32')
x /= 255

out = model.predict(x)
print("prediction:", np.argmax(out))
img

prediction: 2


## creating adversarial examples

onj is the objective function:

$\left\lVert\frac{1}{2}(\tanh(w)+1-x)\right\lVert^2_2 + cf(\frac{1}{2}(\tanh(w)+1)$

where $f(x^{\prime})$ is defined as:

$f(x^{\prime}) = max(max({Z(x^{\prime}_i: i \neq t)} - Z(x^{\prime}_t)), -k)$

get_noise function recovers $\delta$ from w and x:

$\delta = \frac{1}{2}(\tanh(w)+1) - x$

In [8]:
# w: variable, x: original vector, c: tradeoff constant, model: Keras NN, t: target class
def obj(w, x, c, model, t, k=0):
    e = (tf.tanh(w) + 1) * 0.5 - x
    return tf.linalg.norm(e)**2 + c*max(model.predict(x+e, steps=1000).max() - model.predict(x+e, steps=1000)[0][t], -k)

def get_noise(w, x):
    return (np.tanh(w) + 1) * 0.5 - x


## seaching for solution until $C(x+\delta) = t$

In [12]:
x = np.asarray(PIL.Image.open('/Users/zetong/mnist_png/testing/6/11.png'))
img = PIL.Image.fromarray(x, mode='L')
x = x.reshape(28,28)
x = x.reshape(1,28,28,1)
x = x.astype('float32')
x /= 255

e = np.zeros(x.shape)
w = 0.5 * np.log( (2*(e+x)) / (2-2*(e+x)) )
w = tf.Variable(x, dtype=tf.float32)
target = 3

optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(obj(w, x, 0.1, model, target))
print("original: ", np.argmax(model.predict(x)))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        result = sess.run([w,obj(w, x, 0.1, model, target)])
        noise = result[0]
        pert = get_noise(noise, x)
        predict = np.argmax(model.predict(x + pert))
        print(pert)
        print("prediction: ", predict)
        if predict == target:
            break
        sess.run(optimizer)
sess.close()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


original:  6
[[[[ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]]

  [[ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]
   [ 0.5       ]]

  [[ 0.5       ]
   [ 0.5      

[[[[ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]]

  [[ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]
   [ 0.45016605]]

  [[ 0.45016605]
   [ 0.45016605]
   [ 0.450

[[[[ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]]

  [[ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.01535213e-01]
   [ 4.0153

[[[[ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]]

  [[ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]
   [ 0.35521287]]

  [[ 0.35521287]
   [ 0.35521287]
   [ 0.355

...

[[[[ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]]

  [[ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]
   [ 0.02507672]]

  [[ 0.02507672]
   [ 0.02507672]
   [ 0.025

[[[[ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]]

  [[ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.0248087 ]]

  [[ 0.0248087 ]
   [ 0.0248087 ]
   [ 0.024

[[[[ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]]

  [[ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02]
   [ 2.4559021e-02

[[[[ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]]

  [[ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.43258774e-02]
   [ 2.4325

[[[[ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]]

  [[ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.41076648e-02]
   [ 2.4107